In [1]:
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
# !wget -q http://archive.apache.org/dist/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
# !tar -xvf spark-2.4.0-bin-hadoop2.7.tgz
# !pip install -q findspark
# import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"
import findspark
findspark.init()

spark-2.4.0-bin-hadoop2.7/
spark-2.4.0-bin-hadoop2.7/python/
spark-2.4.0-bin-hadoop2.7/python/setup.cfg
spark-2.4.0-bin-hadoop2.7/python/pyspark/
spark-2.4.0-bin-hadoop2.7/python/pyspark/resultiterable.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/python/
spark-2.4.0-bin-hadoop2.7/python/pyspark/python/pyspark/
spark-2.4.0-bin-hadoop2.7/python/pyspark/python/pyspark/shell.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/heapq3.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/join.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/version.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/rdd.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/java_gateway.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/find_spark_home.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/_globals.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/worker.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/accumulators.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/mllib/
spark-2.4.0-bin-hadoop2.7/python/pyspark/mllib/feature.py
spark-2.4.0-bin-hadoop2.7/python/pyspark

In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [3]:
# %cd '/content/gdrive/My Drive/LDS9_K272_ONLINE_TonNuKhanhQuynh/'

/content/gdrive/My Drive/LDS9_K272_ONLINE_TonNuKhanhQuynh


In [4]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import IDF, Tokenizer
from pyspark.ml.feature import NGram

In [5]:
spark = SparkSession.builder.appName('nlp_1').getOrCreate()

In [6]:
import pandas as pd

### Load data

In [7]:
df = pd.read_excel('Du lieu cung cap/womens-ecommerce-clothing-reviews/Womens_Clothing_E_Commerce_Reviews.xlsx')

from pyspark.sql.types import *
df_schema = StructType([StructField("Unnamed: 0", IntegerType(), True)\
                       ,StructField("Clothing ID", IntegerType(), True)\
                       ,StructField("Age", IntegerType(), True)\
                       ,StructField("Title", StringType(), True)\
                       ,StructField("Review Text", StringType(), True)\
                       ,StructField("Rating", IntegerType(), True)\
                       ,StructField("Recommended IND", IntegerType(), True)\
                       ,StructField("Positive Feedback Count", IntegerType(), True)\
                       ,StructField("Division Name", StringType(), True)\
                       ,StructField("Department Name", StringType(), True)\
                       ,StructField("Class Name", StringType(), True)])


df = spark.createDataFrame(df, schema=df_schema)
df.show(3)

+----------+-----------+---+--------------------+--------------------+------+---------------+-----------------------+-------------+---------------+----------+
|Unnamed: 0|Clothing ID|Age|               Title|         Review Text|Rating|Recommended IND|Positive Feedback Count|Division Name|Department Name|Class Name|
+----------+-----------+---+--------------------+--------------------+------+---------------+-----------------------+-------------+---------------+----------+
|         0|        767| 33|                 NaN|Absolutely wonder...|     4|              1|                      0|    Initmates|       Intimate| Intimates|
|         1|       1080| 34|                 NaN|Love this dress! ...|     5|              1|                      4|      General|        Dresses|   Dresses|
|         2|       1077| 60|Some major design...|I had such high h...|     3|              0|                      0|      General|        Dresses|   Dresses|
+----------+-----------+---+------------------

In [8]:
df.count()

23481

In [9]:
df.printSchema()

root
 |-- Unnamed: 0: integer (nullable = true)
 |-- Clothing ID: integer (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Review Text: string (nullable = true)
 |-- Rating: integer (nullable = true)
 |-- Recommended IND: integer (nullable = true)
 |-- Positive Feedback Count: integer (nullable = true)
 |-- Division Name: string (nullable = true)
 |-- Department Name: string (nullable = true)
 |-- Class Name: string (nullable = true)



In [10]:
from pyspark.sql.functions import *

In [11]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+----------+-----------+---+-----+-----------+------+---------------+-----------------------+-------------+---------------+----------+
|Unnamed: 0|Clothing ID|Age|Title|Review Text|Rating|Recommended IND|Positive Feedback Count|Division Name|Department Name|Class Name|
+----------+-----------+---+-----+-----------+------+---------------+-----------------------+-------------+---------------+----------+
|         0|          0|  0| 3810|        845|     0|              0|                      0|           14|             14|        14|
+----------+-----------+---+-----+-----------+------+---------------+-----------------------+-------------+---------------+----------+



In [12]:
print('Before:', df.count())
df = df.where(~isnan('Title'))
df = df.where(~isnan('Review Text'))
# df = df.where(~isnan('Division Name'))
# df = df.where(~isnan('Department Name'))
# df = df.where(~isnan('Class Name'))
print('After:', df.count())

Before: 23481
After: 19670


### Clean and prepare data

In [13]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [14]:
# # Convert categorical strings to index values
# indexer1 = StringIndexer(inputCol='Division Name', outputCol='Division Name_idx')
# indexer2 = StringIndexer(inputCol='Department Name', outputCol='Department Name_idx')
# indexer3 = StringIndexer(inputCol='Class Name', outputCol='Class Name_idx')


# #One-hot encode index values
# onehot1 = OneHotEncoder(
#         inputCols = ['Division Name_idx'],
#         outputCols = ['Division Name_dummy'])

# onehot2 = OneHotEncoder(
#         inputCols = ['Department Name_idx'],
#         outputCols = ['Department Name_dummy'])

# onehot3 = OneHotEncoder(
#         inputCols = ['Class Name_idx'],
#         outputCols = ['Class Name_dummy'])

In [15]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.ml.feature import CountVectorizer, IDF, StringIndexer

In [17]:
# rating_to_class = StringIndexer(inputCol='Rating',outputCol='class')
class_to_num = StringIndexer(inputCol = 'Rating',outputCol = 'label')
tokenizer = Tokenizer(inputCol='Review Text',outputCol='token_Review Text')
stopremove = StopWordsRemover(inputCol='token_Review Text', outputCol="stop_tokens")
count_vec=CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')
idf= IDF(inputCol='c_vec', outputCol='tf_idf')

In [18]:
tokenizer1 = Tokenizer(inputCol='Title',outputCol='token_Title')
stopremove1 = StopWordsRemover(inputCol='token_Title', outputCol="stop_tokens_Title")
count_vec1=CountVectorizer(inputCol='stop_tokens_Title',outputCol='c_vec_Title')
idf1= IDF(inputCol='c_vec_Title', outputCol='tf_idf1')

In [19]:
# Assemble predictors into a single column
clean_up = VectorAssembler(inputCols=['tf_idf1','tf_idf','Recommended IND'], 
                             outputCol='features')


### The model

In [20]:
from pyspark.ml.classification import NaiveBayes

In [21]:
nb = NaiveBayes()

### Pipeline

In [22]:
from pyspark.ml import Pipeline

In [23]:
data_prep_pipe = Pipeline(stages=[
                                  class_to_num,
                                  tokenizer,
                                  stopremove,
                                  count_vec,
                                  idf,
                                  tokenizer1,
                                  stopremove1,
                                  count_vec1,
                                  idf1,
                                  clean_up])

In [24]:
cleaner = data_prep_pipe.fit(df)

In [25]:
clean_data = cleaner.transform(df)

In [26]:
clean_data.show(5)

+----------+-----------+---+--------------------+--------------------+------+---------------+-----------------------+--------------+---------------+----------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Unnamed: 0|Clothing ID|Age|               Title|         Review Text|Rating|Recommended IND|Positive Feedback Count| Division Name|Department Name|Class Name|label|   token_Review Text|         stop_tokens|               c_vec|              tf_idf|         token_Title|   stop_tokens_Title|         c_vec_Title|             tf_idf1|            features|
+----------+-----------+---+--------------------+--------------------+------+---------------+-----------------------+--------------+---------------+----------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------

### Training and Evaluation

In [27]:
clean_data = clean_data.select(['label','features'])

In [28]:
clean_data.show(10)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  2.0|(41394,[45,474,11...|
|  0.0|(41394,[43,471,59...|
|  0.0|(41394,[13,26,591...|
|  3.0|(41394,[100,5911,...|
|  0.0|(41394,[22,1037,4...|
|  1.0|(41394,[253,258,4...|
|  0.0|(41394,[13,5911,5...|
|  0.0|(41394,[20,22,591...|
|  0.0|(41394,[2143,5912...|
|  0.0|(41394,[9,40,5910...|
+-----+--------------------+
only showing top 10 rows



### Resample

In [29]:
df0 = clean_data.filter(col("label") == 0)
df1 = clean_data.filter(col("label") == 1)
df2 = clean_data.filter(col("label") == 2)
df3 = clean_data.filter(col("label") == 3)
df4 = clean_data.filter(col("label") == 4)

In [30]:
def resample(df,combined_df):
    ratio = int(df0.count()/df.count())
    a = range(ratio)
    # duplicate the minority rows
    oversampled = df.withColumn("dummy", explode(array([lit(x) for x in a])))\
                                                                       .drop('dummy')
    # combine both oversampled minority rows and previous majority rows 
    combined_df = combined_df.unionAll(oversampled)
    return combined_df

In [31]:
combined_df = df0
combined_df=resample(df1,combined_df)
combined_df.groupBy("label").count().show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0|10870|
|  1.0| 8576|
+-----+-----+



In [32]:
combined_df=resample(df2,combined_df)
combined_df.groupBy("label").count().show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0|10870|
|  1.0| 8576|
|  2.0| 9852|
+-----+-----+



In [33]:
combined_df=resample(df3,combined_df)
combined_df.groupBy("label").count().show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0|10870|
|  1.0| 8576|
|  3.0| 9513|
|  2.0| 9852|
+-----+-----+



In [34]:
combined_df=resample(df4,combined_df)
combined_df.groupBy("label").count().show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0|10870|
|  1.0| 8576|
|  4.0|10350|
|  3.0| 9513|
|  2.0| 9852|
+-----+-----+



In [35]:
(training,testing) =  combined_df.randomSplit([0.7,0.3])

In [36]:
rating_predictor = nb.fit(training)

In [37]:
test_results = rating_predictor.transform(testing)

In [38]:
test_results.show(10)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(41394,[0,2,14,14...|[-778.11312067982...|[0.99999987902997...|       0.0|
|  0.0|(41394,[0,3,14,33...|[-185.34249998783...|[0.97785096019594...|       0.0|
|  0.0|(41394,[0,3,22,27...|[-1769.3130700084...|[0.99999999999998...|       0.0|
|  0.0|(41394,[0,3,33,52...|[-1374.7209100740...|[1.0,1.4640741767...|       0.0|
|  0.0|(41394,[0,3,34,59...|[-1989.5678643891...|[0.10033448804598...|       1.0|
|  0.0|(41394,[0,3,34,59...|[-989.31186561374...|[0.61771036413331...|       0.0|
|  0.0|(41394,[0,3,101,5...|[-897.26879026213...|[4.75196239255606...|       1.0|
|  0.0|(41394,[0,3,193,5...|[-1383.0816729478...|[1.0,2.6049688956...|       0.0|
|  0.0|(41394,[0,3,235,5...|[-1124.0847158753...|[0.99999999999998...|       0.0|
|  0.0|(41394,[0

In [39]:
test_results.groupBy("label","prediction").count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  2.0|       0.0|   40|
|  1.0|       1.0| 1898|
|  3.0|       2.0|   78|
|  4.0|       2.0|   29|
|  0.0|       1.0|  681|
|  0.0|       4.0|   16|
|  1.0|       0.0|  447|
|  2.0|       2.0| 2666|
|  3.0|       1.0|   15|
|  2.0|       3.0|  114|
|  1.0|       4.0|   15|
|  4.0|       4.0| 3059|
|  2.0|       4.0|   29|
|  3.0|       4.0|   34|
|  2.0|       1.0|  104|
|  1.0|       2.0|  181|
|  0.0|       0.0| 2414|
|  1.0|       3.0|   39|
|  4.0|       3.0|   45|
|  0.0|       2.0|  116|
+-----+----------+-----+
only showing top 20 rows



In [40]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

In [41]:
multi_evaluator = MulticlassClassificationEvaluator()
acc_ = multi_evaluator.evaluate(test_results, {multi_evaluator.metricName:"accuracy"})
acc_

0.8637407157326131

Nhận xét: Kết quả tương đối tốt, nên thử các thuật toán khác để tìm ra thuật toán tốt nhất

## Logistic Regression

In [42]:
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression

In [43]:
lg = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)

In [44]:
predictor_1 = lg.fit(training)

In [45]:
test_results_1 = predictor_1.transform(testing)

In [46]:
test_results_1.groupBy('label', 'prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  2.0|       0.0|   25|
|  1.0|       1.0| 1906|
|  3.0|       2.0|   12|
|  0.0|       1.0|  396|
|  0.0|       4.0|    2|
|  1.0|       0.0|  517|
|  2.0|       2.0| 2801|
|  3.0|       1.0|    1|
|  2.0|       3.0|   48|
|  1.0|       4.0|   14|
|  4.0|       4.0| 3136|
|  2.0|       4.0|   25|
|  3.0|       4.0|    8|
|  2.0|       1.0|   54|
|  1.0|       2.0|  110|
|  0.0|       0.0| 2749|
|  1.0|       3.0|   33|
|  0.0|       2.0|   93|
|  4.0|       0.0|    5|
|  3.0|       3.0| 2864|
+-----+----------+-----+
only showing top 20 rows



In [47]:
acc_eval = MulticlassClassificationEvaluator()
acc_1 = acc_eval.evaluate(test_results_1)
print("Accuracy of model at predicting: {}".format(acc_1))

Accuracy of model at predicting: 0.9072929632774678


### Random Forest

In [61]:
rf = RandomForestClassifier(labelCol="label", \
                            featuresCol="features",\
                            numTrees = 500, \
                            maxDepth = 5, \
                            maxBins = 64)

In [62]:
predictor_2 = rf.fit(training)

In [63]:
test_results_2 = predictor_2.transform(testing)

In [64]:
test_results_2.groupBy('label', 'prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  2.0|       0.0| 1849|
|  3.0|       2.0|   31|
|  4.0|       2.0|    5|
|  0.0|       4.0|    7|
|  1.0|       0.0| 2510|
|  2.0|       2.0|  113|
|  2.0|       3.0|    2|
|  1.0|       4.0|   64|
|  4.0|       4.0| 2864|
|  2.0|       4.0|  989|
|  3.0|       4.0| 1863|
|  1.0|       2.0|    6|
|  0.0|       0.0| 3243|
|  0.0|       2.0|    1|
|  4.0|       0.0|  272|
|  3.0|       3.0|   17|
|  3.0|       0.0|  974|
+-----+----------+-----+



In [65]:
test_results_2.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|       0.0| 8848|
|       4.0| 5787|
|       3.0|   19|
|       2.0|  156|
+----------+-----+



In [66]:
acc_eval = MulticlassClassificationEvaluator()
acc_2 = acc_eval.evaluate(test_results_2)
print("Accuracy of model at predicting: {}".format(acc_2))

Accuracy of model at predicting: 0.27052117616439586


#### Nhận xét: Trong các thuật toán trên, logistic regression cho kết quả tốt nhất

### Predict new data

In [67]:
data_new = df = pd.read_excel('Du lieu cung cap/womens-ecommerce-clothing-reviews/Womens_Clothing_E_Commerce_Reviews.xlsx', sheet_name='new_reviews')


data_new = spark.createDataFrame(data_new)
data_new.show(3)

+----------+-----------+---+--------------------+--------------------+---------------+-----------------------+-------------+---------------+----------+
|Unnamed: 0|Clothing ID|Age|               Title|         Review Text|Recommended IND|Positive Feedback Count|Division Name|Department Name|Class Name|
+----------+-----------+---+--------------------+--------------------+---------------+-----------------------+-------------+---------------+----------+
|         0|       1077| 53|Dress looks like ...|Dress runs small ...|              0|                     14|      General|        Dresses|   Dresses|
|         1|        862| 66|            Cute top|Nice top. armhole...|              1|                      2|      General|           Tops|     Knits|
|         2|       1080| 31|        Underwhelmed|Was really excite...|              0|                      1|      General|        Dresses|   Dresses|
+----------+-----------+---+--------------------+--------------------+---------------+--

In [68]:
data_prep_pipe_new = Pipeline(stages=[
                                  tokenizer,
                                  stopremove,
                                  count_vec,
                                  idf,
                                  tokenizer1,
                                  stopremove1,
                                  count_vec1,
                                  idf1,
                                  clean_up])

In [69]:
unlabeled_data = data_prep_pipe_new.fit(data_new).transform(data_new)

In [70]:
unlabeled_data.show(5)

+----------+-----------+---+--------------------+--------------------+---------------+-----------------------+-------------+---------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Unnamed: 0|Clothing ID|Age|               Title|         Review Text|Recommended IND|Positive Feedback Count|Division Name|Department Name|Class Name|   token_Review Text|         stop_tokens|               c_vec|              tf_idf|         token_Title|   stop_tokens_Title|         c_vec_Title|             tf_idf1|            features|
+----------+-----------+---+--------------------+--------------------+---------------+-----------------------+-------------+---------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------

In [71]:
unlabeled_data = unlabeled_data.select("features")

In [72]:
prediction_new = predictor_1.transform(unlabeled_data)

In [78]:
prediction_new.show()

Py4JJavaError: ignored